In [1]:
import pandas as pd

In [3]:
# 저장한 CSV 피일 경로 
file_path = 'cleaned_circle_chart_data.csv'

# CSV 파일 불러오기
df = pd.read_csv(file_path)

In [4]:
df.head()

,date,Artist
0,2023-09-24,AKMU (악뮤)
1,2023-09-24,"다이나믹 듀오 (Dynamic Duo), 이영지"
2,2023-09-24,NewJeans
3,2023-09-24,정국
4,2023-09-24,전소미


In [5]:
import re

In [6]:
# 괄호 안의 영문 제거 후 한글만 남기기
df['Artist'] = df['Artist'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())

In [7]:
# 데이터 확인 
df.head()

,date,Artist
0,2023-09-24,AKMU
1,2023-09-24,"다이나믹 듀오 , 이영지"
2,2023-09-24,NewJeans
3,2023-09-24,정국
4,2023-09-24,전소미


In [8]:
# 아티스트를 쉼표로 분리하고 개별 행으로 나누기
df_expanded = df.assign(Artist=df['Artist'].str.split(', ')).explode('Artist').reset_index(drop=True)

# 데이터 확인
df_expanded.head()


,date,Artist
0,2023-09-24,AKMU
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,NewJeans
4,2023-09-24,정국


In [9]:
# 날짜와 아티스트 컬럼에서 중복되는 행이 있는지 확인
duplicates_check = df_expanded.duplicated(subset=['date', 'Artist']).sum()

# 중복된 행이 있으면 숫자로 표시됩니다.
print(f'중복된 행 개수: {duplicates_check}')


중복된 행 개수: 309


In [10]:
# 최종 데이터 검토
df_expanded.head()  # 최종 데이터가 제대로 처리되었는지 확인합니다.


,date,Artist
0,2023-09-24,AKMU
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,NewJeans
4,2023-09-24,정국


In [11]:
artist_name_mapping = {
    'AKMU': '악뮤',
    'NewJeans': '뉴진스',
    'Dynamic Duo': '다이나믹 듀오',
    '화사': '화사',
    'IVE': '아이브',
    'Woody': '우디',
    'STAYC': '스테이씨',
    'LE SSERAFIM': '르세라핌',
    'Lauv': '라우브',
    'aespa': '에스파',
    'BIG Naughty': '서동현',
    'Paul Blanco': '폴 블랑코',
    'RIIZE': '라이즈',
    'ITZY': '있지',
    'Charlie Puth': '찰리 푸스',
    'V': '뷔',
    'Melomance': '멜로망스',
    'Crush': '크러쉬',
    'H1-KEY': '하이키',
    'JISOO': '지수',
    'NCT U': 'NCT U',
    'The Kid LAROI': '더 키드 라로이',
    'Justin Bieber': '저스틴 비버',
    'Agust D': '어거스트 디',
    'EXO': '엑소',
    'BLACKPINK': '블랙핑크',
    'Charlie Puth, 정국, 방탄소년단': '찰리 푸스, 정국, 방탄소년단',
    # 필요한 아티스트 계속 추가
}


In [12]:
def convert_artist_name(artist_name):
    for eng_name, kor_name in artist_name_mapping.items():
        artist_name = artist_name.replace(eng_name, kor_name)
    return artist_name.strip()

# 아티스트 이름 변환 적용
df_expanded['Artist'] = df_expanded['Artist'].apply(convert_artist_name)

# 변환 결과 확인
df_expanded.head()


,date,Artist
0,2023-09-24,악뮤
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,뉴진스
4,2023-09-24,정국


In [13]:
# 영어로 된 아티스트 이름이 있는지 확인
eng_artists_check = df_expanded[df_expanded['Artist'].apply(lambda x: bool(re.search(r'[a-zA-Z]', x)))]

# 영어로 된 아티스트가 있는지 확인
if len(eng_artists_check) > 0:
    print("영어로 된 아티스트 이름이 존재합니다:")
    print(eng_artists_check)
else:
    print("영어로 된 아티스트 이름이 없습니다.")


영어로 된 아티스트 이름이 존재합니다:
            date           Artist
23    2023-09-24               DK
35    2023-09-24            NCT U
48    2023-09-24      FIFTY FIFTY
50    2023-09-24               DK
51    2023-09-24        NCT DREAM
...          ...              ...
6122  2024-09-08        Sam Smith
6126  2024-09-08           Sondia
6130  2024-09-08  Alaina Castillo
6132  2024-09-08               YB
6136  2024-09-08    Kenshi Yonezu

[1454 rows x 2 columns]


In [14]:
# 같은 날짜에 중복된 아티스트 이름 확인
duplicate_artists_check = df_expanded[df_expanded.duplicated(subset=['date', 'Artist'], keep=False)]

# 중복된 아티스트가 있는지 확인
if len(duplicate_artists_check) > 0:
    print("같은 날짜에 중복된 아티스트 이름이 존재합니다:")
    print(duplicate_artists_check)
else:
    print("같은 날짜에 중복된 아티스트 이름이 없습니다.")


같은 날짜에 중복된 아티스트 이름이 존재합니다:
            date   Artist
1     2023-09-24  다이나믹 듀오
4     2023-09-24       정국
23    2023-09-24       DK
37    2023-09-24   저스틴 비버
50    2023-09-24       DK
...          ...      ...
6101  2024-09-08      임한별
6103  2024-09-08      순순희
6112  2024-09-08      #안녕
6116  2024-09-08   저스틴 비버
6118  2024-09-08       허각

[594 rows x 2 columns]


In [15]:
# 영어 이름을 한글로 변환하는 사전 추가
artist_name_mapping = {
    'AKMU (악뮤)': '악뮤',
    'NewJeans': '뉴진스',
    '화사(Hwa Sa)': '화사',
    'IVE (아이브)': '아이브',
    'Woody': '우디',
    'STAYC(스테이씨)': '스테이씨',
    'LE SSERAFIM (르세라핌)': '르세라핌',
    'Lauv': '라우브',
    'aespa': '에스파',
    'BIG Naughty (서동현)': '서동현',
    'Paul Blanco': '폴 블랑코',
    'RIIZE': '라이즈',
    'ITZY (있지)': '있지',
    'DK': '디케이',
    'Charlie Puth': '찰리 푸스',
    'V': '뷔',
    'Melomance': '멜로망스',
    'Crush': '크러쉬',
    'H1-KEY': '하이키',
    'JISOO': '지수',
    'NCT U': '엔시티 유',
    'The Kid LAROI, Justin Bieber': '더 키드 라로이, 저스틴 비버',
    'FIFTY FIFTY': '피프티 피프티',
    'NCT DREAM': '엔시티 드림',
    'Agust D': '어거스트 디',
    'Doja Cat': '도자 캣',
    'BLACKPINK': '블랙핑크',
    'Justin Bieber': '저스틴 비버',
    'Sam Smith': '샘 스미스',
    'Ed Sheeran': '에드 시런',
    'EXO': '엑소',
    'Billie Eilish': '빌리 아일리시',
    'Taylor Swift': '테일러 스위프트',
    'Stray Kids': '스트레이 키즈',
    'ATEEZ': '에이티즈',
    'TREASURE': '트레저',
    'Ariana Grande': '아리아나 그란데',
    'Mariah Carey': '머라이어 캐리',
    'VIVIZ': '비비지',
    'BTOB': '비투비',
    'ITZY': '있지',
    'ENHYPEN': '엔하이픈',
    'Charlie Puth, 정국, 방탄소년단': '찰리 푸스, 정국, 방탄소년단',
    # 필요한 추가 변환
}

# 아티스트 이름을 한글로 변환하는 함수
def convert_artist_name(artist_name):
    for eng_name, kor_name in artist_name_mapping.items():
        artist_name = artist_name.replace(eng_name, kor_name)
    return artist_name.strip()

# 아티스트 이름 변환 적용
df['Artist'] = df['Artist'].apply(convert_artist_name)

# 변환된 데이터 확인
df.head()


,date,Artist
0,2023-09-24,AKMU
1,2023-09-24,"다이나믹 듀오 , 이영지"
2,2023-09-24,뉴진스
3,2023-09-24,정국
4,2023-09-24,전소미


In [16]:
# 아티스트를 쉼표로 분리하고 개별 행으로 나누기
df_expanded = df.assign(Artist=df['Artist'].str.split(', ')).explode('Artist').reset_index(drop=True)

# 결과 확인
df_expanded.head()


,date,Artist
0,2023-09-24,AKMU
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,뉴진스
4,2023-09-24,정국


In [17]:
# 아티스트 이름에 영어가 포함된 경우 확인
eng_artists_check = df_expanded[df_expanded['Artist'].apply(lambda x: bool(re.search(r'[a-zA-Z]', x)))]

# 영어로 된 아티스트가 있는지 확인
if len(eng_artists_check) > 0:
    print("영어로 된 아티스트 이름이 존재합니다:")
    print(eng_artists_check)
else:
    print("영어로 된 아티스트 이름이 없습니다.")


영어로 된 아티스트 이름이 존재합니다:
            date           Artist
0     2023-09-24             AKMU
8     2023-09-24              I뷔E
10    2023-09-24            STAYC
12    2023-09-24      LE SSERAFIM
17    2023-09-24      BIG Naughty
...          ...              ...
6126  2024-09-08           Sondia
6130  2024-09-08  Alaina Castillo
6132  2024-09-08               YB
6135  2024-09-08      BIG Naughty
6136  2024-09-08    Kenshi Yonezu

[1372 rows x 2 columns]


In [18]:
# 영어로 된 아티스트 이름이 있는지 확인
eng_artists_check = df_expanded[df_expanded['Artist'].apply(lambda x: bool(re.search(r'[a-zA-Z]', x)))]

# 남아 있는 영어 아티스트 이름 목록 확인
eng_artists_list = eng_artists_check['Artist'].unique()
eng_artists_list


array(['AKMU', 'I뷔E', 'STAYC', 'LE SSERAFIM', 'BIG Naughty', '10cm',
       'Zac Efron', 'Zendaya', 'BOYNEXTDOOR', 'YOASOBI', 'DAY6', 'j-hope',
       ' J. Cole', 'imase', 'Christopher', 'DJ Premier', 'KISS OF LIFE',
       'Anne-Marie', 'WSG워너비', 'Yyeon', 'ZEROBASEONE', 'Sondia',
       'Dan + Shay', 'NMIXX', 'Yuuri', 'BE`O', 'NCT 127',
       'League of Legends', 'dori', 'TOIL', 'Kid Wine', 'The Kid LAROI',
       'Central Cee', 'PLA뷔E', 'Sia', 'Kelly Clarkson', 'Alessia Cara',
       'DEAN', 'QWER', 'SG워너비', 'BABYMONSTER', 'Ava Max',
       'Straight No Chaser', 'Wham!', '뷔I뷔IZ', 'Michael Buble',
       'Jonas Brothers', 'ASH ISLAND', 'TWICE', 'Brenda Lee',
       'John Legend', 'PITTA', '2am', 'SF9', 'YB', 'YENA', 'KCM', 'TWS',
       'J.Tajor', 'WOOZI', 'AB6IX', 'meenoi', 'I.M', 'Ayumu Imazu',
       'Benson Boone', 'CRA뷔ITY', 'Natalie Jane', 'Peder Elias',
       'Men`s Tear', 'Alaina Castillo', 'HIGH4', 'LUCY', 'Creepy Nuts',
       'MAX', 'SUPER JUNIOR-D&E', 'BIGBANG', 'Shawn M

In [22]:
artist_name_mapping.update({
    'AKMU': '악뮤',
    'I뷔E': '아이브',  # IVE 오타로 추정하여 '아이브'로 수정
    'STAYC': '스테이씨',
    'LE SSERAFIM': '르세라핌',
    'BIG Naughty': '서동현',
    '10cm': '10cm',  # 이미 적절한 이름이므로 유지
    'Zac Efron': '잭 에프론',
    'Zendaya': '젠데이아',
    'BOYNEXTDOOR': '보이넥스트도어',
    'YOASOBI': '요아소비',
    'DAY6': '데이식스',
    'j-hope': '제이홉',
    'J. Cole': '제이콜',
    'imase': '이마세',
    'Christopher': '크리스토퍼',
    'DJ Premier': '디제이 프리미어',
    'KISS OF LIFE': '키스 오브 라이프',
    'Anne-Marie': '앤 마리',
    'WSG워너비': 'WSG워너비',  # 이미 적절한 이름이므로 유지
    'Yyeon': '이연',
    'ZEROBASEONE': '제로베이스원',
    'Sondia': '손디아',
    'Dan + Shay': '댄 앤 셰이',
    'NMIXX': '엔믹스',
    'Yuuri': '유우리',
    'BE`O': '비오',
    'NCT 127': '엔시티 127',
    'League of Legends': '리그 오브 레전드',
    'dori': '도리',
    'TOIL': '토일',
    'Kid Wine': '키드 와인',
    'The Kid LAROI': '더 키드 라로이',
    'Central Cee': '센트럴 씨',
    'PLAVE': '플레이브',
    'Sia': '시아',
    'Kelly Clarkson': '켈리 클락슨',
    'Alessia Cara': '알레시아 카라',
    'DEAN': '딘',
    'QWER': '큐더블유이알',
    'SG워너비': 'SG워너비',  # 이미 적절한 이름이므로 유지
    'BABYMONSTER': '베이비몬스터',
    'Ava Max': '에이바 맥스',
    'Straight No Chaser': '스트레이트 노 체이서',
    'Wham!': '왬',
    'VIVIZ': '비비지',
    'Michael Buble': '마이클 부블레',
    'Jonas Brothers': '조나스 브라더스',
    'ASH ISLAND': '애쉬 아일랜드',
    'TWICE': '트와이스',
    'Brenda Lee': '브렌다 리',
    'John Legend': '존 레전드',
    'PITTA': '피타',
    '2am': '2am',  # 이미 적절한 이름이므로 유지
    'SF9': '에스에프나인',
    'YB': '윤도현 밴드',
    'YENA': '최예나',
    'KCM': '케이씨엠',
    'TWS': '투어스',
    'J.Tajor': '제이 타조',
    'WOOZI': '우지',
    'AB6IX': '에이비식스',
    'meenoi': '미노이',
    'I.M': '아이엠',
    'Ayumu Imazu': '아유무 이마즈',
    'Benson Boone': '벤슨 분',
    'CRAVITY': '크래비티',
    'Natalie Jane': '나탈리 제인',
    'Peder Elias': '페더 엘리아스',
    'Men`s Tear': '맨스 티어',
    'Alaina Castillo': '알라이나 카스틸로',
    'HIGH4': '하이포',
    'LUCY': '루시',
    'Creepy Nuts': '크리피 넛츠',
    'MAX': '맥스',
    'SUPER JUNIOR-D&E': '슈퍼주니어 D&E',
    'BIGBANG': '빅뱅',
    'Shawn Mendes': '숀 멘데스',
    'RM': '알엠',
    'Andy Grammer': '앤디 그래머',
    'tripleS': '트리플에스',
    'Sabrina Carpenter': '사브리나 카펜터',
    'Gist': '지스트',
    'HYNN': '박혜원',
    'XIA': '시아',
    '정한 X 원우': '정한 X 원우',
    'Tommy Richman': '토미 리치맨',
    '뷔aundy': '바운디',
    'NCT WISH': '엔시티 위시',
    'Sunset Rollercoaster 落日飛車': '선셋 롤러코스터',
    'Seiko Matsuda': '세이코 마츠다',
    'Sofia Carson': '소피아 카슨',
    'S.Papa': '에스 파파',
    'Lady Gaga': '레이디 가가',
    'Bruno Mars': '브루노 마스',
    'KIXO': '키조',
    'MEOVV': '미야오',
    'Kenshi Yonezu': '요네즈 켄시'
})


In [23]:
# 아티스트 이름 변환 적용
df_expanded['Artist'] = df_expanded['Artist'].apply(convert_artist_name)

# 변환된 데이터 확인
df_expanded.head()


,date,Artist
0,2023-09-24,악뮤
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,뉴진스
4,2023-09-24,정국


In [24]:
# 아티스트 이름에 영어가 포함된 경우 확인
eng_artists_check = df_expanded[df_expanded['Artist'].apply(lambda x: bool(re.search(r'[a-zA-Z]', x)))]

# 영어로 된 아티스트가 있는지 확인
if len(eng_artists_check) > 0:
    print("영어로 된 아티스트 이름이 존재합니다:")
    print(eng_artists_check)
else:
    print("영어로 된 아티스트 이름이 없습니다.")


영어로 된 아티스트 이름이 존재합니다:
            date  Artist
60    2023-09-24    10cm
98    2023-09-24  WSG워너비
191   2023-10-01    10cm
229   2023-10-01  WSG워너비
321   2023-10-08    10cm
...          ...     ...
6061  2024-09-08   PLA뷔E
6065  2024-09-08   MEO뷔뷔
6071  2024-09-08   뷔I뷔IZ
6086  2024-09-08    10cm
6097  2024-09-08  NCT 위시

[155 rows x 2 columns]


In [25]:
# 영어로 된 아티스트 이름이 있는지 확인
eng_artists_check = df_expanded[df_expanded['Artist'].apply(lambda x: bool(re.search(r'[a-zA-Z]', x)))]

# 남아 있는 영어 아티스트 이름 목록 확인
eng_artists_list = eng_artists_check['Artist'].unique()
eng_artists_list


array(['10cm', 'WSG워너비', 'PLA뷔E', 'SG워너비', '뷔I뷔IZ', '2am', 'CRA뷔ITY',
       '슈퍼주니어 D&E', '정한 X 원우', 'NCT 위시', 'MEO뷔뷔'], dtype=object)

In [28]:
artist_name_mapping.update({
    'PLA뷔E': '플레이브',
    '뷔I뷔IZ': '비비지',
    'CRA뷔ITY': '크래비티',
    '슈퍼주니어 D&E': '슈퍼주니어 D&E',
    'NCT 위시': '엔시티 WISH',
    'MEO뷔뷔': '미야오',  # 임시로 '미야오'로 변환 (추후 확인 필요)
})

# 다시 변환 적용
df_expanded['Artist'] = df_expanded['Artist'].apply(convert_artist_name)

# 변환된 데이터 확인
df_expanded.head()


,date,Artist
0,2023-09-24,악뮤
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,뉴진스
4,2023-09-24,정국


In [29]:
# 영어로 된 아티스트 이름이 있는지 확인
eng_artists_check = df_expanded[df_expanded['Artist'].apply(lambda x: bool(re.search(r'[a-zA-Z]', x)))]

# 남아 있는 영어 아티스트 이름 목록 확인
eng_artists_list = eng_artists_check['Artist'].unique()
eng_artists_list


array(['10cm', 'WSG워너비', 'SG워너비', '2am', '슈퍼주니어 D&E', '정한 X 원우'],
      dtype=object)

In [30]:
# 날짜별 중복된 아티스트 확인
duplicate_artists_check = df_expanded[df_expanded.duplicated(subset=['date', 'Artist'], keep=False)]

# 중복된 아티스트가 있는지 확인
if len(duplicate_artists_check) > 0:
    print("날짜별로 중복된 아티스트 이름이 존재합니다:")
    print(duplicate_artists_check)
else:
    print("날짜별로 중복된 아티스트 이름이 없습니다.")


날짜별로 중복된 아티스트 이름이 존재합니다:
            date   Artist
1     2023-09-24  다이나믹 듀오
4     2023-09-24       정국
23    2023-09-24      디케이
37    2023-09-24   저스틴 비버
50    2023-09-24      디케이
...          ...      ...
6101  2024-09-08      임한별
6103  2024-09-08      순순희
6112  2024-09-08      #안녕
6116  2024-09-08   저스틴 비버
6118  2024-09-08       허각

[594 rows x 2 columns]


In [31]:
# 날짜별로 중복된 아티스트 이름 제거 (첫 번째 중복만 남기고 나머지 제거)
df_unique_artists = df_expanded.drop_duplicates(subset=['date', 'Artist'], keep='first')

# 중복 제거 후 결과 확인
df_unique_artists.head()


,date,Artist
0,2023-09-24,악뮤
1,2023-09-24,다이나믹 듀오
2,2023-09-24,이영지
3,2023-09-24,뉴진스
4,2023-09-24,정국


In [32]:
# 다시 중복 여부 확인
duplicate_artists_check = df_unique_artists[df_unique_artists.duplicated(subset=['date', 'Artist'], keep=False)]

if len(duplicate_artists_check) == 0:
    print("중복된 아티스트 이름이 없습니다.")
else:
    print("중복된 아티스트 이름이 여전히 존재합니다.")


중복된 아티스트 이름이 없습니다.


In [33]:
# 중복이 제거된 파일을 CSV로 저장
file_path = 'unique_artists_by_date.csv'  # 원하는 파일 경로로 수정할 수 있습니다.
df_unique_artists.to_csv(file_path, index=False)

print(f"파일이 '{file_path}'로 저장되었습니다.")


파일이 'unique_artists_by_date.csv'로 저장되었습니다.
